In [1]:
import pandas as pd
import sympy

In [2]:
math_train = pd.read_json('number_word_std.dev.json')

In [83]:
math_train.iloc[33:43,:]

,ans,ans_simple,equations,id,index,sources,text
33,11; 13,"[11, 13]","[unkn: 2*k-1, 2*k+1, equ: 3(2*k-1)=7+2*(2*k+1)]",algebra.com.797657,868,algebra.com.797657,find two consecutive odd integers such that th...
34,{11; 3},"[11, 3]","[unkn: x,y, equ: x + y = 14, equ: x^2 + y^2 = ...",yahoo.answers.20061106190512aaqso2e,1381,https://answers.yahoo.com/question/index?qid=2...,The sum of two numbers is fourteen. The sum of...
35,22; 24; 26,"[22, 24, 26]","[unkn: 2*x,2*x+2,2*x+4, equ: 2(2*x+2+2*x+4)-34...",algebra.com.2530,35,algebra.com.2530,Find three consecutive even integers such that...
36,{3; 7},"[3, 7]","[unkn: x,y, equ: x+y=10, equ: 3*y+3=8*x]",algebra.com.2819,36,"[algebra.com.2819, algebra.com.24638, algebra....",The sum of two integers is 10. Three times the...
37,{12; 17},"[12, 17]","[unkn: x,y, equ: x+y=29, equ: 8*x=5*y+11]",algebra.com.457023,610,algebra.com.457023,The sum of two integers is twenty-nine. Eight ...
38,{52; 73},"[52, 73]","[unkn: x,y, equ: x + y = 125, equ: 5*x - 3*y =...",yahoo.answers.20090223121444aar9wxc,1566,https://answers.yahoo.com/question/index?qid=2...,The sum of two numbers is 125. Five times one ...
39,ans_no_result,[],"[unkn: n,n+7, equ: n+(n+7) = 84]",algebra.com.367618,507,algebra.com.367618,find two consecutive multiples of 7 whose sum ...
40,92; 93,"[92, 93]","[unkn: x,x+1, equ: x+(x+1)=185]",algebra.com.449815,606,algebra.com.449815,the sum of two consecutive integers is 185. Fi...
41,90,[90],"[unkn: x, equ: x/3+60=x/15+84]",algebra.com.3318,42,algebra.com.3318,Sixty more than a number divided by three is e...
42,17; 18; 19,"[17, 18, 19]","[unkn: k-1,k,k+1, equ: 2*k=(k+1)+17]",algebra.com.100249,44,algebra.com.100249,Find 3 consecutive numbers such that twice the...


In [84]:
def is_linear(original_string_equation):
    """ Checks if an equation is linear"""
    try:    
        # Strips the 'equ:"
        strip_equ = original_string_equation.lstrip("equ: ")
        # Finds the variables
        list_of_var = []
        variables = ""
        for char in strip_equ:
            if char in "abcdefghijklmnopqrstuvwxyxABCDEFGHIJKLMNOPQRSTUVWXYZ":
                if char not in list_of_var:
                    variables = variables + char + " "
                    list_of_var.append(char)
        # Generates an Eq object for separation of the sides of equation
        eq_S = sympy.Eq(*map(sympy.S, strip_equ.split("=")))
        # Combines two sides of the equation
        final_version = eq_S.args[0] + eq_S.args[1]*-1
        # Put it in a list so that the 'linear_eq_to_matrix' can handle
        if final_version.args_cnc()[0] == []:
            final_as_list = final_version.args_cnc()[1]
        else:
            final_as_list = final_version.args_cnc()[0]
        symb = sympy.symbols(variables)
        A, b = sympy.linear_eq_to_matrix(final_as_list, symb)
        if not any(list(A)) and list(b)[0] != 0:
            return False
        else:
            return True
    except:
        return False

In [85]:
def is_set_linear(list_of_equations):
    all_linear = True
    list_without_unkn = list_of_equations[1:]
    if not all(map(is_linear, list_without_unkn)):
        return False
    else:
        return True

In [86]:
is_set_linear(math_train["equations"][82])

False

In [88]:
math_train["is_linear"] = math_train["equations"].map(is_set_linear)

In [89]:
math_train.head()

,ans,ans_simple,equations,id,index,sources,text,is_linear
0,{40;29},"[40, 29]","[unkn: x,y, equ: x=y+11, equ: 3*x=4*y+4]",algebra.com.117395,1043,algebra.com.117395,one number is 11 more than another number. Fin...,True
1,6; 9,"[6, 9]","[unkn: x,y, equ: x + 3 = y, equ: 2*y + 12 = 5*x]",yahoo.answers.20091227181039aazhen8,1617,https://answers.yahoo.com/question/index?qid=2...,One number is 3 less than a second number. Twi...,True
2,{34; 28},"[34, 28]","[unkn: x,y, equ: x+y=62, equ: x-y=6]",algebra.com.101793,49,algebra.com.101793,Find two numbers whose sum is 62 and whose dif...,True
3,{42;26},"[42, 26]","[unkn: x,y, equ: x+y=68, equ: x-y=16]",algebra.com.112903,1032,algebra.com.112903,the sum of two numbers is 68. their difference...,True
4,{77; 20},"[77, 20]","[unkn: x,y, equ: x+y=97, equ: x-y=57]",algebra.com.193198,1256,algebra.com.193198,the sum of two numbers is 97. the difference o...,True


In [90]:
math_train["is_linear"].value_counts()

True     278
False     96
Name: is_linear, dtype: int64